In [1]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person
from tqdm import tqdm
import json
import numpy as np

ImportError: No module named 'requests'

In [3]:
auth_file = "/home/ivan/pascal_adversarial_faces/azure_auth.json"
with open(auth_file, "r") as f:
    auth_data = json.loads(f.read())


In [4]:
face_client = FaceClient(
    auth_data["endpoint"], 
    CognitiveServicesCredentials(auth_data["key"])
)

In [31]:
# face_client.person_group.delete(person_group_id="robust_community_naive_mean_0_6_0p5")

In [6]:
def measure_azure_recall(
    face_client,
    image_directory,
    azure_person_group_name
):
    
    person_id_to_name = {}
    identities = []
    for p in face_client.person_group_person.list(person_group_id=azure_person_group_name):
        person_id_to_name[p.person_id] = p.name
        identities.append(p.name)

    discovery = []
    true = []
    identified_as = []

    for protector in tqdm(identities):
        # We are sourcing query photos from epsilon_0.0.
        # In those cases, all subfolders in the "protected" identity have the same, clean
        # photo of the protector, so we just pick any single one that exists (e.g. n000958)
        # For the case where n000958 is itself the protector, n000958 is not present in its protected
        # subfolders, so we pick n000029 without loss of generality.
        if protector == "n000958":
            protected = "n000029"
        else:
            protected = "n000958"

        query_photos_paths = sorted(glob.glob(
            f"{image_directory}/{protector}/community_naive_mean/{protected}/epsilon_0.0/png/*"
        ))
        
        # For this person group, we picked the first 10 lexicographically sorted
        # photos to use in the lookup set, so for the query, we use the 11th and on.
        # (The query is not supposed to be in the lookup set).
        for i in range(11, len(query_photos_paths)):
            faces_in_query_photos = face_client.face.detect_with_stream(
                open(query_photos_paths[i], "r+b"), 
                detectionModel='detection_02'
            )
            if len(faces_in_query_photos) > 0:
                break


        # There should only be one face, so we use that as the query face.
        results = face_client.face.identify(
            [faces_in_query_photos[0].face_id],
            azure_person_group_name
        )
        
        true.append(protector)

        if len(results) < 1 or len(results[0].candidates) < 1:
            discovery.append(0.0)
            identified_as.append("None")

        else: 
            top_identity = person_id_to_name[results[0].candidates[0].person_id]

            identified_as.append(top_identity)
            
            # Note the switch of the term protector here:
            # protectors are also protected but we call them protectors because of the folder structure
            # In this case, the query photo belongs to the protector -- who is also protected by decoys 
            # of *other* protectors. Therefore, if the identity returned is that of the "protector,"
            # this is a failure in the defense.
            if top_identity == protector:
                discovery.append(1.0)
            else:
                discovery.append(0.0)

        time.sleep(10)
    
    for true_id, recognized_id in zip(true, identified_as):
        print(f"Face of {true_id} identitifed as {recognized_id}")
        
    return sum(discovery)/len(discovery)

In [10]:
def measure_azure_recall_for_random_5_images(
    face_client,
    image_directory,
    azure_person_group_name
):
    
    person_id_to_name = {}
    identities = []
    for p in face_client.person_group_person.list(person_group_id=azure_person_group_name):
        person_id_to_name[p.person_id] = p.name
        identities.append(p.name)

    discovery = []
    true = []
    identified_as = []

    for protector in tqdm(identities):
        # We are sourcing query photos from epsilon_0.0.
        # In those cases, all subfolders in the "protected" identity have the same, clean
        # photo of the protector, so we just pick any single one that exists (e.g. n000958)
        # For the case where n000958 is itself the protector, n000958 is not present in its protected
        # subfolders, so we pick n000029 without loss of generality.
        if protector == "n000958":
            protected = "n000029"
        else:
            protected = "n000958"

        query_photos_paths = sorted(glob.glob(
            f"{image_directory}/{protector}/community_naive_mean/{protected}/epsilon_0.0/png/*"
        ))
        
        # For this person group, we picked the first 10 lexicographically sorted
        # photos to use in the lookup set, so for the query, we use the 11th and on.
        # (The query is not supposed to be in the lookup set).
        for i in np.random.choice(len(query_photos_paths), 5):
            faces_in_query_photos = face_client.face.detect_with_stream(
                open(query_photos_paths[i], "r+b"), 
                detectionModel='detection_02'
            )
            if len(faces_in_query_photos) < 1:
                continue


            # There should only be one face, so we use that as the query face.
            results = face_client.face.identify(
                [faces_in_query_photos[0].face_id],
                azure_person_group_name
            )

            true.append(protector)

            if len(results) < 1 or len(results[0].candidates) < 1:
                discovery.append(0.0)
                identified_as.append("None")

            else: 
                top_identity = person_id_to_name[results[0].candidates[0].person_id]

                identified_as.append(top_identity)

                # Note the switch of the term protector here:
                # protectors are also protected but we call them protectors because of the folder structure
                # In this case, the query photo belongs to the protector -- who is also protected by decoys 
                # of *other* protectors. Therefore, if the identity returned is that of the "protector,"
                # this is a failure in the defense.
                if top_identity == protector:
                    discovery.append(1.0)
                else:
                    discovery.append(0.0)

            time.sleep(15)

    for true_id, recognized_id in zip(true, identified_as):
        print(f"Face of {true_id} identitifed as {recognized_id}")
        
    return sum(discovery)/len(discovery)

In [25]:
measure_azure_recall(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_10_5_0p5"
)



  0%|          | 0/19 [00:00<?, ?it/s]

  5%|▌         | 1/19 [00:10<03:07, 10.43s/it]

 11%|█         | 2/19 [00:20<02:56, 10.37s/it]

 16%|█▌        | 3/19 [00:30<02:45, 10.34s/it]

 21%|██        | 4/19 [00:41<02:34, 10.32s/it]

 26%|██▋       | 5/19 [00:51<02:23, 10.28s/it]

 32%|███▏      | 6/19 [01:01<02:13, 10.28s/it]

 37%|███▋      | 7/19 [01:12<02:04, 10.35s/it]

 42%|████▏     | 8/19 [01:22<01:53, 10.31s/it]

 47%|████▋     | 9/19 [01:32<01:42, 10.30s/it]

 53%|█████▎    | 10/19 [01:42<01:32, 10.28s/it]

 58%|█████▊    | 11/19 [01:53<01:22, 10.27s/it]

 63%|██████▎   | 12/19 [02:03<01:11, 10.27s/it]

 68%|██████▊   | 13/19 [02:13<01:01, 10.25s/it]

 74%|███████▎  | 14/19 [02:23<00:51, 10.27s/it]

 79%|███████▉  | 15/19 [02:34<00:41, 10.30s/it]

 84%|████████▍ | 16/19 [02:44<00:30, 10.31s/it]

 89%|████████▉ | 17/19 [02:54<00:20, 10.29s/it]

 95%|█████████▍| 18/19 [03:05<00:10, 10.35s/it]

100%|██████████| 19/19 [03:15<00:00, 10.30s/it]

Face of n002763 identitifed as n002763
Face of n007548 identitifed as n007548
Face of n004658 identitifed as n004658
Face of n002503 identitifed as None
Face of n009232 identitifed as n009232
Face of n002647 identitifed as n002647
Face of n000958 identitifed as n000958
Face of n003356 identitifed as n003356
Face of n001683 identitifed as n001683
Face of n009288 identitifed as None
Face of n008613 identitifed as n008613
Face of n005303 identitifed as n005303
Face of n008655 identitifed as None
Face of n001781 identitifed as n001781
Face of n005359 identitifed as None
Face of n003215 identitifed as None
Face of n000029 identitifed as n000029
Face of n009114 identitifed as None
Face of n005427 identitifed as n005427


0.6842105263157895

In [27]:
measure_azure_recall_for_all_images(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_10_5_0p5"
)



  0%|          | 0/19 [00:00<?, ?it/s]

APIErrorException: (429) Requests to the Face - Detect Operation under Face API - v1.0 have exceeded rate limit of your current Face F0 pricing tier. Please retry after 9 seconds. To increase your rate limit switch to a paid tier.

In [16]:
measure_azure_recall(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_1_1_0p5"
)



  0%|          | 0/19 [00:00<?, ?it/s]

  5%|▌         | 1/19 [00:10<03:04, 10.24s/it]

 11%|█         | 2/19 [00:20<02:53, 10.23s/it]

 16%|█▌        | 3/19 [00:30<02:43, 10.22s/it]

 21%|██        | 4/19 [00:40<02:33, 10.21s/it]

 26%|██▋       | 5/19 [00:51<02:23, 10.21s/it]

 32%|███▏      | 6/19 [01:01<02:13, 10.24s/it]

 37%|███▋      | 7/19 [01:11<02:03, 10.26s/it]

 42%|████▏     | 8/19 [01:21<01:52, 10.25s/it]

 47%|████▋     | 9/19 [01:32<01:42, 10.26s/it]

 53%|█████▎    | 10/19 [01:42<01:32, 10.24s/it]

 58%|█████▊    | 11/19 [01:52<01:21, 10.24s/it]

 63%|██████▎   | 12/19 [02:02<01:11, 10.24s/it]

 68%|██████▊   | 13/19 [02:13<01:01, 10.25s/it]

 74%|███████▎  | 14/19 [02:23<00:51, 10.26s/it]

 79%|███████▉  | 15/19 [02:33<00:41, 10.26s/it]

 84%|████████▍ | 16/19 [02:43<00:30, 10.26s/it]

 89%|████████▉ | 17/19 [02:54<00:20, 10.26s/it]

 95%|█████████▍| 18/19 [03:04<00:10, 10.25s/it]

100%|██████████| 19/19 [03:14<00:00, 10.24s/it]

Face of n000029 identitifed as n000029
Face of n002763 identitifed as n002763
Face of n000958 identitifed as n000958
Face of n005359 identitifed as n005359
Face of n004658 identitifed as n004658
Face of n009232 identitifed as n009232
Face of n005303 identitifed as n005303
Face of n001683 identitifed as n001683
Face of n009288 identitifed as n009288
Face of n007548 identitifed as n007548
Face of n008613 identitifed as n008613
Face of n008655 identitifed as None
Face of n003356 identitifed as n003356
Face of n001781 identitifed as n001781
Face of n003215 identitifed as n003215
Face of n009114 identitifed as n009114
Face of n002503 identitifed as n002503
Face of n002647 identitifed as n002647
Face of n005427 identitifed as n005427


0.9473684210526315

In [30]:
measure_azure_recall(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_2_6_0p5"
)




  0%|          | 0/19 [00:00<?, ?it/s]


  5%|▌         | 1/19 [00:10<03:05, 10.31s/it]


 11%|█         | 2/19 [00:20<02:54, 10.29s/it]


 16%|█▌        | 3/19 [00:30<02:44, 10.27s/it]


 21%|██        | 4/19 [00:41<02:35, 10.35s/it]


 26%|██▋       | 5/19 [00:51<02:25, 10.36s/it]


 32%|███▏      | 6/19 [01:01<02:14, 10.31s/it]


 37%|███▋      | 7/19 [01:12<02:03, 10.29s/it]


 42%|████▏     | 8/19 [01:22<01:53, 10.28s/it]


 47%|████▋     | 9/19 [01:32<01:43, 10.31s/it]


 53%|█████▎    | 10/19 [01:43<01:33, 10.37s/it]


 58%|█████▊    | 11/19 [01:53<01:23, 10.43s/it]


 63%|██████▎   | 12/19 [02:04<01:12, 10.36s/it]


 68%|██████▊   | 13/19 [02:14<01:02, 10.34s/it]


 74%|███████▎  | 14/19 [02:24<00:51, 10.32s/it]


 79%|███████▉  | 15/19 [02:34<00:41, 10.31s/it]


 84%|████████▍ | 16/19 [02:45<00:30, 10.29s/it]


 89%|████████▉ | 17/19 [02:55<00:20, 10.28s/it]


 95%|█████████▍| 18/19 [03:05<00:10, 10.28s/it]


100%|██████████| 19/19 [03:15<00:00, 10.31s/it]

Face of n005427 identitifed as n005427
Face of n003215 identitifed as n003215
Face of n005303 identitifed as n005303
Face of n008613 identitifed as n008613
Face of n009232 identitifed as n009232
Face of n008655 identitifed as None
Face of n009114 identitifed as None
Face of n001781 identitifed as n001781
Face of n004658 identitifed as n004658
Face of n007548 identitifed as n007548
Face of n002647 identitifed as n002647
Face of n001683 identitifed as n001683
Face of n003356 identitifed as n003356
Face of n009288 identitifed as n009288
Face of n005359 identitifed as n005359
Face of n000029 identitifed as n000029
Face of n002763 identitifed as n002763
Face of n000958 identitifed as n000958
Face of n002503 identitifed as n002503


0.8947368421052632

In [5]:
measure_azure_recall(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_2_1_0p5"
)

100%|██████████| 19/19 [03:16<00:00, 10.34s/it]

Face of n008655 identitifed as None
Face of n001683 identitifed as n001683
Face of n003215 identitifed as n003215
Face of n004658 identitifed as n004658
Face of n007548 identitifed as n007548
Face of n001781 identitifed as n001781
Face of n000029 identitifed as n000029
Face of n008613 identitifed as n008613
Face of n009114 identitifed as n009114
Face of n000958 identitifed as n000958
Face of n002763 identitifed as n002763
Face of n005303 identitifed as n005303
Face of n005427 identitifed as None
Face of n009288 identitifed as None
Face of n005359 identitifed as n005359
Face of n003356 identitifed as n003356
Face of n002503 identitifed as n002503
Face of n002647 identitifed as n002647
Face of n009232 identitifed as n009232


0.8421052631578947

In [15]:
def increment_dict(dictionary, key):
    if key not in dictionary:
        dictionary[key] = 1
    else:
        dictionary[key] += 1
    return dictionary
    
def read_log_file(log_file_path, base_images_path):
    identity_to_num_total = {}
    identity_to_num_clean = {}
    identity_to_num_decoys_for_others = {}
    identity_to_num_decoys_by_others = {}
    
    with open(log_file_path, "r") as f:
        curr_id = None
        for line in f:
            if line.startswith("n"):
                curr_id = line.strip("\n")
            else:
                increment_dict(identity_to_num_total, curr_id)

                # clean base path, e.g. /data/vggface/test_perturbed
                # then strip \n characters
                line = line[len(base_images_path) + 1:].strip("\n") 
                # split by path separator
                line = line.split("/")
                # format is base_path/true_id/attack_strategy/target/epsilon_X/other_stuff
                true_id = line[0]
                target_id = line[2]
                epsilon = line[3]
                assert true_id == curr_id, f"In line {' '.join(line)} mismatch between true_id {true_id} and current id {curr_id}"

                if epsilon == "epsilon_0.0":
                    increment_dict(identity_to_num_clean, curr_id)
                elif epsilon.startswith("epsilon_"):
                    increment_dict(identity_to_num_decoys_for_others, curr_id)
                else:
                    raise Exception(f"Invalid epsilon {epsilon}")

                increment_dict(identity_to_num_decoys_by_others, target_id)
        
        for key, value in identity_to_num_total.items():
            print(f"Identity {key} has {value} total images associated with it")
        
        for key, value in identity_to_num_clean.items():
            print(f"Identity {key} has {value} clean images")
        
        for key, value in identity_to_num_decoys_for_others.items():
            print(f"Identity {key} has {value} decoys for others")
            
        for key, value in identity_to_num_decoys_by_others.items():
            print(f"Others have provided  {value} decoys for identity {key}")
                

In [16]:
read_log_file(
    "/home/ivan/pascal_adversarial_faces/azure_face_logfiles/robust_community_naive_mean_2_1_0p5.txt",
    "/data/vggface/test_perturbed_sampled"
)

Identity n001781 has 18 total images associated with it
Identity n009232 has 20 total images associated with it
Identity n000958 has 13 total images associated with it
Identity n003356 has 20 total images associated with it
Identity n008655 has 17 total images associated with it
Identity n008613 has 14 total images associated with it
Identity n004658 has 19 total images associated with it
Identity n001683 has 20 total images associated with it
Identity n002647 has 12 total images associated with it
Identity n009288 has 15 total images associated with it
Identity n005427 has 15 total images associated with it
Identity n002763 has 16 total images associated with it
Identity n002503 has 18 total images associated with it
Identity n003215 has 10 total images associated with it
Identity n005359 has 13 total images associated with it
Identity n005303 has 13 total images associated with it
Identity n007548 has 15 total images associated with it
Identity n000029 has 15 total images associated 

In [8]:
measure_azure_recall(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_0_6_0p5"
)


100%|██████████| 19/19 [03:24<00:00, 10.76s/it]

Face of n009114 identitifed as n009114
Face of n009288 identitifed as n009288
Face of n008613 identitifed as n008613
Face of n005427 identitifed as n005427
Face of n004658 identitifed as n004658
Face of n007548 identitifed as n007548
Face of n003356 identitifed as n003356
Face of n009232 identitifed as n009232
Face of n005359 identitifed as None
Face of n000958 identitifed as n000958
Face of n008655 identitifed as None
Face of n002647 identitifed as n002647
Face of n003215 identitifed as n003215
Face of n001683 identitifed as n001683
Face of n002763 identitifed as n002763
Face of n002503 identitifed as n002503
Face of n001781 identitifed as n001781
Face of n000029 identitifed as n000029
Face of n005303 identitifed as n005303


0.8947368421052632

In [9]:
for g in face_client.person_group.list():
    print(g.name)

community_naive_mean_10_5_0p1
community_naive_mean_49_0_0p1
epsilon_0d1_flip_bright_gaussian
epsilon_0d1_flip_crop_resize_bright_gaussian
epsilon_0d25_flip_bright_gaussian
epsilon_0d25_flip_crop_resize_bright_gaussian
epsilon_0d5_flip_bright_gaussian
epsilon_0d5_flip_crop_resize_bright_gaussian
ivan_yoshi
ivan_yoshi_pascal
ivan_yoshi_strong_attack
plain
robust_community_naive_mean_0_1_0p5
robust_community_naive_mean_0_6_0p5
robust_community_naive_mean_10_5_0p5
robust_community_naive_mean_1_1_0p5
robust_community_naive_mean_2_1_0p5
robust_community_naive_mean_2_5_0p5
robust_community_naive_mean_2_6_0p5
something_random


In [13]:
measure_azure_recall_for_random_5_images(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_0_6_0p5"
)

  5%|▌         | 1/19 [01:01<18:21, 61.20s/it]

Face of n009114 identitifed as n009114
Face of n009114 identitifed as n009114
Face of n009114 identitifed as n009114
Face of n009114 identitifed as None


 11%|█         | 2/19 [02:02<17:21, 61.28s/it]

Face of n009114 identitifed as n009114
Face of n009114 identitifed as n009114
Face of n009114 identitifed as n009114
Face of n009114 identitifed as None
Face of n009288 identitifed as n009288
Face of n009288 identitifed as n009288
Face of n009288 identitifed as n009288
Face of n009288 identitifed as None


 16%|█▌        | 3/19 [03:19<17:33, 65.83s/it]

...

 95%|█████████▍| 18/19 [19:12<01:05, 65.29s/it]

Face of n009114 identitifed as n009114
Face of n009114 identitifed as n009114
Face of n009114 identitifed as n009114
Face of n009114 identitifed as None
Face of n009288 identitifed as n009288
Face of n009288 identitifed as n009288
Face of n009288 identitifed as n009288
Face of n009288 identitifed as None
Face of n008613 identitifed as n008613
Face of n008613 identitifed as n008613
Face of n008613 identitifed as n008613
Face of n008613 identitifed as n008613
Face of n008613 identitifed as n008613
Face of n005427 identitifed as n005427
Face of n005427 identitifed as None
Face of n005427 identitifed as None
Face of n005427 identitifed as n005427
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n00754

100%|██████████| 19/19 [19:58<00:00, 63.10s/it]

Face of n009114 identitifed as n009114
Face of n009114 identitifed as n009114
Face of n009114 identitifed as n009114
Face of n009114 identitifed as None
Face of n009288 identitifed as n009288
Face of n009288 identitifed as n009288
Face of n009288 identitifed as n009288
Face of n009288 identitifed as None
Face of n008613 identitifed as n008613
Face of n008613 identitifed as n008613
Face of n008613 identitifed as n008613
Face of n008613 identitifed as n008613
Face of n008613 identitifed as n008613
Face of n005427 identitifed as n005427
Face of n005427 identitifed as None
Face of n005427 identitifed as None
Face of n005427 identitifed as n005427
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n00754

0.8461538461538461

In [14]:
measure_azure_recall_for_random_5_images(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_2_1_0p5"
)

  5%|▌         | 1/19 [00:46<13:51, 46.21s/it]

Face of n008655 identitifed as n008655
Face of n008655 identitifed as n008655
Face of n008655 identitifed as n008655


 11%|█         | 2/19 [02:03<15:42, 55.46s/it]

Face of n008655 identitifed as n008655
Face of n008655 identitifed as n008655
Face of n008655 identitifed as n008655
Face of n001683 identitifed as n005303
Face of n001683 identitifed as n001683
Face of n001683 identitifed as n001683
Face of n001683 identitifed as n001683
Face of n001683 identitifed as None


 16%|█▌        | 3/19 [03:04<15:17, 57.32s/it]

...

 95%|█████████▍| 18/19 [19:56<01:12, 72.30s/it]

Face of n008655 identitifed as n008655
Face of n008655 identitifed as n008655
Face of n008655 identitifed as n008655
Face of n001683 identitifed as n005303
Face of n001683 identitifed as n001683
Face of n001683 identitifed as n001683
Face of n001683 identitifed as n001683
Face of n001683 identitifed as None
Face of n003215 identitifed as None
Face of n003215 identitifed as n003215
Face of n003215 identitifed as None
Face of n003215 identitifed as n003215
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as None
Face of n004658 identitifed as n004658
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n001781 identitifed as n001781
Face of n001781 identitifed as n001781
Face of n001781 identitifed as n001781
Face of n001781 identitifed as n001781
Face of n001781 identitifed as None
F

100%|██████████| 19/19 [20:57<00:00, 66.21s/it]

Face of n008655 identitifed as n008655
Face of n008655 identitifed as n008655
Face of n008655 identitifed as n008655
Face of n001683 identitifed as n005303
Face of n001683 identitifed as n001683
Face of n001683 identitifed as n001683
Face of n001683 identitifed as n001683
Face of n001683 identitifed as None
Face of n003215 identitifed as None
Face of n003215 identitifed as n003215
Face of n003215 identitifed as None
Face of n003215 identitifed as n003215
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as None
Face of n004658 identitifed as n004658
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n001781 identitifed as n001781
Face of n001781 identitifed as n001781
Face of n001781 identitifed as n001781
Face of n001781 identitifed as n001781
Face of n001781 identitifed as None
F

0.7926829268292683

In [15]:
measure_azure_recall_for_random_5_images(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_10_5_0p5"
)

  5%|▌         | 1/19 [00:46<13:48, 46.03s/it]

Face of n002763 identitifed as n002763
Face of n002763 identitifed as n002763
Face of n002763 identitifed as n002763


 11%|█         | 2/19 [02:02<15:37, 55.16s/it]

Face of n002763 identitifed as n002763
Face of n002763 identitifed as n002763
Face of n002763 identitifed as n002763
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548


 16%|█▌        | 3/19 [03:19<16:27, 61.73s/it]

...

 95%|█████████▍| 18/19 [19:54<01:02, 62.26s/it]

Face of n002763 identitifed as n002763
Face of n002763 identitifed as n002763
Face of n002763 identitifed as n002763
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n004658 identitifed as n004658
Face of n004658 identitifed as None
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n002503 identitifed as None
Face of n002503 identitifed as None
Face of n002503 identitifed as n002503
Face of n002503 identitifed as n002503
Face of n002503 identitifed as n002503
Face of n009232 identitifed as n009232
Face of n009232 identitifed as n009232
Face of n009232 identitifed as n009232
Face of n009232 identitifed as n009232
Face of n009232 identitifed as n009232
Face of n002647 identitifed as n002647
Face of n002647 identitifed as n002647
Face of n002647 identitifed as n00

100%|██████████| 19/19 [21:11<00:00, 66.90s/it]

Face of n002763 identitifed as n002763
Face of n002763 identitifed as n002763
Face of n002763 identitifed as n002763
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n007548 identitifed as n007548
Face of n004658 identitifed as n004658
Face of n004658 identitifed as None
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n004658 identitifed as n004658
Face of n002503 identitifed as None
Face of n002503 identitifed as None
Face of n002503 identitifed as n002503
Face of n002503 identitifed as n002503
Face of n002503 identitifed as n002503
Face of n009232 identitifed as n009232
Face of n009232 identitifed as n009232
Face of n009232 identitifed as n009232
Face of n009232 identitifed as n009232
Face of n009232 identitifed as n009232
Face of n002647 identitifed as n002647
Face of n002647 identitifed as n002647
Face of n002647 identitifed as n00

0.8072289156626506

In [ ]:
def train(face_client, person_group_name):
    print()
    print(f'Training the person group {person_group_name}')
    # Train the person group
    face_client.person_group.train(person_group_name)

    while (True):
        training_status = face_client.person_group.get_training_status(person_group_name)
    print("Training status: {}.".format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        print('Training the person group has failed.')
        break
    time.sleep(5)